# Решение SAS

## Прогнозирование вероятности невозврата кредита

### Данные ООО «Хоум Кредит энд Финанс Банк» 

2017, Александр Дьяконов

https://alexanderdyakonov.wordpress.com/


## Описание метода

* предобработка данных
* подготовка признаковых матриц
 * одна матрица - статистики по всем кредитам (среднее, максимум, минимум и т.д. исходных признаков)
 * вторая - аналогично, но перед вычислением статистик информация по одинаковым кредитам схлопывается
* ответ - среднее модели на этих двух признаковых пространствах
* модель - среднее 4х xgb и 4х lgb (бустинги над деревьями разных реализаций)
* модель запускается 15 раз, ответ получается усреднением

**Время работы** скрипта: 36 минут (вычисление признаков) + 4.2 часа (обучение модели)

**Важно**: код для Python 3.x

In [1]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np

%pylab inline
plt.style.use('seaborn-dark')

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

import matplotlib as mpl

plt.rc('font', size=14)

Populating the interactive namespace from numpy and matplotlib


### Загрузка данных

In [2]:
import lightgbm as lgb

# обучение
data_train = pd.read_csv('train.csv')
# контроль
data_test = pd.read_csv('test.csv')

print ('размеры', data_train.shape, data_test.shape)

# целевой вектор
data = data_train.groupby('ID')['DEF'].mean().reset_index()
y = data.DEF.values
del data_train['DEF']

print ('объектов в обучении', len(y))

размеры (1787571, 34) (1665298, 33)
объектов в обучении 135155


### Порождение признакового пространства

сначала перечень всех используемых функций...

In [3]:
# всё в одну таблицу
data_all = pd.concat([data_train, data_test])
data_all.shape

(3452869, 33)

In [4]:
def make_time_features(data):
    """
    сделать временные признаки
    """

    data['SK_DATE_DECISION'] = pd.to_datetime(data['SK_DATE_DECISION'], format='%Y%m%d')
    data['DTIME_CREDIT'] = pd.to_datetime(data['DTIME_CREDIT'], format='%d.%m.%Y')
    data['DTIME_CREDIT_ENDDATE'] = pd.to_datetime(data['DTIME_CREDIT_ENDDATE'], format='%d.%m.%Y')
    data['DTIME_CREDIT_ENDDATE_FACT'] = pd.to_datetime(data['DTIME_CREDIT_ENDDATE_FACT'], format='%d.%m.%Y')
    data['DTIME_CREDIT_UPDATE'] = pd.to_datetime(data['DTIME_CREDIT_UPDATE'], format='%d.%m.%Y')
    
    data['SK_DATE_DECISION_dayofweek'] = data['SK_DATE_DECISION'].dt.dayofweek
    data['SK_DATE_DECISION_day'] = data['SK_DATE_DECISION'].dt.day
    
    
    data['DTIME_CREDIT_dayofweek'] = data['DTIME_CREDIT'].dt.dayofweek
    data['DTIME_CREDIT_ENDDATE_FACT_dayofweek'] = data['DTIME_CREDIT_ENDDATE_FACT'].dt.dayofweek
    data['DTIME_CREDIT_UPDATE_dayofweek'] = data['DTIME_CREDIT_UPDATE'].dt.dayofweek
    
    
    # на какой срок
    data['DELTA_CREDIT'] = (data['DTIME_CREDIT_ENDDATE'] - data['DTIME_CREDIT']).astype("timedelta64[D]")
    data['DELTA_CREDIT'] = data['DELTA_CREDIT'].fillna(2158.9)
    data['DELTA_CREDIT2'] = (data['DTIME_CREDIT_ENDDATE_FACT'] - data['DTIME_CREDIT']).astype("timedelta64[D]")
    data['DELTA_CREDIT2'] = data['DELTA_CREDIT2'].fillna(410.5)
    # взять их отношения
    data['DELTA_ENDS'] = (data['DTIME_CREDIT_ENDDATE'] - data['DTIME_CREDIT_ENDDATE_FACT']).astype("timedelta64[D]")
    data['DELTA_ENDS'] = data['DELTA_ENDS'].fillna(600.8)
    
    
    data['DIV_DELTAS'] = (data['DELTA_CREDIT2'] + 0.0) / (data['DELTA_CREDIT'].abs() + 0.1)
    
    # сколько дней назад открыт кредит
    data['DTIME_CREDIT']  = (data['SK_DATE_DECISION'] - data['DTIME_CREDIT'] ).astype("timedelta64[D]") # .astype(int)
    data['DTIME_CREDIT_ENDDATE'] = (data['SK_DATE_DECISION'] - data['DTIME_CREDIT_ENDDATE'] ).astype("timedelta64[D]")
    
    # +
    data['null_in_DTIME_CREDIT_ENDDATE'] = data['DTIME_CREDIT_ENDDATE'].isnull() + 0
    data['DTIME_CREDIT_ENDDATE'] = data['DTIME_CREDIT_ENDDATE'].fillna(-823)
    
    data['DTIME_CREDIT_ENDDATE_FACT'] = (data['SK_DATE_DECISION'] - data['DTIME_CREDIT_ENDDATE_FACT'] ).astype("timedelta64[D]")
    data['null_in_DTIME_CREDIT_ENDDATE_FACT'] = data['DTIME_CREDIT_ENDDATE_FACT'].isnull() + 0
    data['DTIME_CREDIT_ENDDATE_FACT'] = data['DTIME_CREDIT_ENDDATE_FACT'].fillna(532)    
    
    
    # обновление в источнике
    data['DTIME_CREDIT_UPDATE'] = (data['SK_DATE_DECISION'] - data['DTIME_CREDIT_UPDATE'] ).astype("timedelta64[D]")
    
    tm = data['SK_DATE_DECISION'].values
    
    ##########del data['SK_DATE_DECISION'] # ОСТАВИТЬ

    
    return (data, tm)


def make_str_features(data):
    """
    перекодировать строковые признаки
    """    
    
    data['CREDIT_CURRENCY'] = data['CREDIT_CURRENCY'].map({'rur': 0, 'usd': 1, 'eur': 2, 'chf': 2})
    
    data['CREDIT_FACILITY'] = data['CREDIT_FACILITY'].fillna(-1.0).astype(int)
    
    data['CREDIT_TYPE'] = data['CREDIT_TYPE'].map({0: 0,
                                                   1: 1, #  кредит на автомобиль
                                                   2: 0,
                                                   3: 3, # ипотека
                                                   4: 4, # кредитная карта
                                                   5: 5, # потребительский кредит
                                                   6: 0,
                                                   7: 0,
                                                   8: 0,
                                                   9: 0,
                                                   10: 0,
                                                   11: 0,
                                                   12: 0,
                                                   13: 0,
                                                   14: 0,
                                                   15: 0,
                                                   16: 0,
                                                   17: 0,
                                                   18: 0,
                                                   19: 19, # микрозайм
                                                   90: 0,
                                                   99: 0
                                                  })
    
    data['DOLG'] = (data['AMT_CREDIT_SUM_DEBT'] + 0.0) / (data['AMT_CREDIT_SUM'].abs() + 1.0)

    return (data)

def make_null_features(data):
    """
    заменить пропуски
    """
    # в make_str_features data['CREDIT_FACILITY'] = data['CREDIT_FACILITY'].fillna(-1) # [ nan,   9.,   0.,   2.,   1.] мелк. кат
    data['AMT_CREDIT_SUM'] = data['AMT_CREDIT_SUM'].fillna(85202.1) # слишком мало значений
    data['null_in_AMT_CREDIT_SUM_DEBT'] = data.AMT_CREDIT_SUM_DEBT.isnull() + 0
    data['AMT_CREDIT_SUM_DEBT'] = data['AMT_CREDIT_SUM_DEBT'].fillna(26895.1) # но тут едет значение
    data['null_in_AMT_CREDIT_SUM_LIMIT'] = data.AMT_CREDIT_SUM_LIMIT.isnull() + 0
    data['AMT_CREDIT_SUM_LIMIT'] = data['AMT_CREDIT_SUM_LIMIT'].fillna(1558.7) # но тут едет значение
    data['null_in_AMT_ANNUITY'] = data.AMT_ANNUITY.isnull() + 0
    data['AMT_ANNUITY'] = data['AMT_ANNUITY'].fillna(2882.1) # но тут едет значение
    
    data['null_in_AMT_CREDIT_MAX_OVERDUE'] = data.AMT_CREDIT_MAX_OVERDUE.isnull() + 0
    data['AMT_CREDIT_MAX_OVERDUE'] = data['AMT_CREDIT_MAX_OVERDUE'].fillna(751.7) 
    
    return (data)

In [5]:
user_features = ['AMT_REQ_SOURCE_HOUR',
                 'AMT_REQ_SOURCE_DAY',
                 'AMT_REQ_SOURCE_WEEK',
                 'AMT_REQ_SOURCE_MON',
                 'AMT_REQ_SOURCE_QRT',
                 'AMT_REQ_SOURCE_YEAR',
                 'SK_DATE_DECISION_dayofweek',
                 'SK_DATE_DECISION_day',
                 ### 'SK_DATE_DECISION' # NEW!!!
                ]


def make_user_features(data, user_features):
    """
    признаки, которые зависят только от пользователя
    """
    tmp = data[ ['ID'] + user_features]
    tmp.fillna(-999, inplace=True) # можно не делать...
    tmp = tmp.groupby('ID').last() # .mean()
    tmp.reset_index(inplace=True)
    return (tmp)

bin_features = ['CREDIT_SUM_TYPE', # бинарный
                'null_in_AMT_CREDIT_SUM_DEBT',
                'null_in_AMT_CREDIT_SUM_LIMIT',
                'null_in_AMT_ANNUITY',
                'null_in_DTIME_CREDIT_ENDDATE',
                'null_in_DTIME_CREDIT_ENDDATE_FACT'
               ]

def make_bin_features(df, names):
    """
    бинарные признаки -> средние значения
    """
    tmp = df.groupby('ID')[names].mean().fillna(-1)
    tmp.columns = ['mean__' + str(x) for x in tmp.columns]
    tmp = tmp.reset_index()
    return (tmp)

cat_features = ['NUM_SOURCE', # источник
                'CREDIT_TYPE', # тип договора
                'CREDIT_ACTIVE', # статус
                'CREDIT_CURRENCY', # валюта
                'CREDIT_FACILITY', # ???
                'DTIME_CREDIT_dayofweek',
                'DTIME_CREDIT_ENDDATE_FACT_dayofweek',
                'DTIME_CREDIT_UPDATE_dayofweek'
               ]

def make_cat_feature(df, name, normalize=True):
    """
    один категориальный признак
    в матрицу числа вхождений
    """
    tmp = df.groupby(['ID', name]).size().unstack().fillna(0)
    tmp.columns = [name + '__' + str(x) for x in tmp.columns]
    if normalize:
        tmp = tmp.div(tmp.sum(axis=1), axis=0)
        tmp.fillna(0.0)
    tmp = tmp.reset_index()
    return (tmp)

def make_cat_features(df, cat_features, normalize=True):
    """
    обработать все категориальные признаки
    """
    tmp = df.groupby('ID').size().reset_index()
    tmp.columns = ['ID', 'SIZE']
    for name in cat_features:
        tmp2 = make_cat_feature(df, name, normalize)
        print ('category', name, tmp2.shape)
        tmp = tmp.merge(tmp2, how='left', on='ID')
        
    return (tmp)

log_features = ['LOG_CREDIT_DAY_OVERDUE', # текущая просроченная задолженность, дни
                'LOG_CNT_CREDIT_PROLONG', # число пролонгаций кредита
                'LOG_AMT_CREDIT_SUM', # сумма кредита (LOG ????)
                'LOG_AMT_CREDIT_SUM_DEBT', # сумма оставшегося долга (DIV ????)
                'LOG_AMT_CREDIT_SUM_LIMIT', # лимит (для карт)
                'LOG_AMT_CREDIT_SUM_OVERDUE', # текущая просроченная задолженность, сумма
                'LOG_AMT_CREDIT_MAX_OVERDUE', # макс задолженность
                'LOG_CREDIT_DELAY5',
                'LOG_CREDIT_DELAY30',
                'LOG_CREDIT_DELAY60',
                'LOG_CREDIT_DELAY90',
                'LOG_CREDIT_DELAY_MORE',
                'LOG_AMT_ANNUITY',
                'LOG_DOLG']

def make_logs(df, log_features):
    """
    прологарифмировать признаки
    """
    for name in log_features:
        df[name] = np.log(df[name[4:]].abs() + 1) # 'LOG_' +
    return df

real_features = ['CREDIT_DAY_OVERDUE', # текущая просроченная задолженность, дни
                 'CNT_CREDIT_PROLONG', # число пролонгаций кредита
                 'AMT_CREDIT_SUM', # сумма кредита (LOG ????)
                 'AMT_CREDIT_SUM_DEBT', # сумма оставшегося долга (DIV ????)
                 'AMT_CREDIT_SUM_LIMIT', # лимит (для карт)
                 'AMT_CREDIT_SUM_OVERDUE', # текущая просроченная задолженность, сумма
                 'AMT_CREDIT_MAX_OVERDUE', # макс задолженность
                 'CREDIT_DELAY5',
                 'CREDIT_DELAY30',
                 'CREDIT_DELAY60',
                 'CREDIT_DELAY90',
                 'CREDIT_DELAY_MORE',
                 'AMT_ANNUITY',
                 'DOLG']

realtime_features = ['DELTA_CREDIT',
                     'DELTA_CREDIT2',
                     'DELTA_ENDS',
                     'DTIME_CREDIT',
                     'DTIME_CREDIT_ENDDATE',
                     'DTIME_CREDIT_ENDDATE_FACT',
                     'DTIME_CREDIT_UPDATE',
                     'DIV_DELTAS']


    
def make_real_feature(df, name):
    """
    вычисление разных статистик
    """
    tmp = df.groupby('ID')[name].agg({mean, max, min, median, sum, var}).fillna(-1)
    tmp.columns = [name + '__' + str(x) for x in tmp.columns]
    tmp = tmp.reset_index()
    return (tmp)

def make_real_features(df, real_features):
    """
    обработать все вещественные признаки
    """
    tmp = df.groupby('ID').size().reset_index()
    tmp.columns = ['ID', 'SIZE']
    for name in real_features:
        tmp2 = make_real_feature(df, name,)
        print ('real', name, tmp2.shape)
        tmp = tmp.merge(tmp2, how='left', on='ID')
    del tmp['SIZE']
    return (tmp)

def count_in_str(x, ch, deg=1):
    """
    веса для анализа строки
    """
    return (sum((1./t ** deg for t, c in enumerate(x, start=1) if c==ch)) / (0.01 + sum((1./t ** deg for t in range(1, 1+len(x))))))

txt_features = ['TXT_0', 'TXT_1', 'TXT_2', 'TXT_3', 'TXT_4', 'TXT_5', 'TXT_X', 'TXT_C']
txt_features2 = ['TXTn_0', 'TXTn_1', 'TXTn_2', 'TXTn_3', 'TXTn_4', 'TXTn_5', 'TXTn_X', 'TXTn_C']
txt_features0 = ['TXT_L']

txt_features3 = ['BF_0', 'BF_1', 'BF_2', 'BF_3', 'BF_4', 'BF_5', 'BF_X', 'BF_C',
                 'BF2_0', 'BF2_1', 'BF2_2', 'BF2_3', 'BF2_4', 'BF2_5', 'BF2_X', 'BF2_C',
                 'BF0_0', 'BF0_1', 'BF0_2', 'BF0_3', 'BF0_4', 'BF0_5', 'BF0_X', 'BF0_C'
                ]


def make_txt(tt):
    """
    сделать признаки по
    платёжной строке
    """
    
    dct = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, 'X':6, 'C':7}
    
    tt['TEXT_PAYMENT_DISCIPLINE'].fillna('', inplace=True)
    
    chs = ['0', '1', '2', '3', '4', '5', 'X', 'C']
    for ch in chs:
        tt['TXT_' + ch] = tt['TEXT_PAYMENT_DISCIPLINE'].map(lambda x: x.count(ch))
        tt['BF_' + ch] = tt['TEXT_PAYMENT_DISCIPLINE'].map(lambda x: count_in_str(x, ch, 1))
        tt['BF2_' + ch] = tt['TEXT_PAYMENT_DISCIPLINE'].map(lambda x: count_in_str(x, ch, 2))
        tt['BF0_' + ch] = tt['TEXT_PAYMENT_DISCIPLINE'].map(lambda x: count_in_str(x, ch, 0.5))
        print (ch, 'ждите...')
        
    
    tt['TXT_L'] = tt['TEXT_PAYMENT_DISCIPLINE'].map(len)
    
    tt['TPD0'] = data_all['TEXT_PAYMENT_DISCIPLINE'].map(lambda x: dct[x[0]] if len(x)>0 else -1)
        
    tt[txt_features2] = tt[txt_features].div(tt[txt_features].abs().sum(axis=1) + 0.001, axis=0)
    
    
    return (tt)

### обработка признаков

* перекодировка
* обработка строк
* заполнение пропусков

In [6]:
from time import time
time_ = time()

data_all, time_all = make_time_features(data_all)
data_all = make_str_features(data_all)
data_all = make_null_features(data_all)
### data_all = make_logs(data_all, log_features)
data_all = make_txt(data_all)

print ('время: ', time() - time_)

0
1
2
3
4
5
X
C
время:  2065.846471309662


In [7]:
# схлопнуть информацию об одинаковых кредитах...
data_pre = data_all.drop(['SK_DATE_DECISION', 'TEXT_PAYMENT_DISCIPLINE'], axis=1).groupby(['ID', 'CREDIT_CURRENCY', 'DTIME_CREDIT','CREDIT_TYPE']).max().reset_index()

### Сформировать признаковую матрицу

Строим 2 матрицы

* по всей информации
* по "схлопнутой" из *data_pre*

In [8]:
def make_feature_matrix(train_fm, ids):
    """
    Создать признаковую матрицу
    """
    data = pd.DataFrame({'ID': ids})
    
    for t, tmp in enumerate(train_fm):
        tmp.columns = [str(t) + 'c_' + x if x!='ID' else 'ID' for x in tmp.columns]
        data = data.merge(tmp, on='ID', how='left') #, inplace='True')
    
    return (data)

In [9]:
# 1я версия
data_fm = [make_user_features(data_all, user_features),
            make_cat_features(data_all, cat_features + ['TPD0'], normalize=True),
            make_bin_features(data_all, bin_features),
            make_real_features(data_all, real_features),
            make_real_features(data_all, realtime_features),
            make_real_features(data_all, txt_features + txt_features2 + txt_features0),
            make_real_features(data_all, txt_features3)
           ]

data = make_feature_matrix(data_fm, data_all.groupby('ID').size().reset_index()['ID'].values)

dataA2 = data[data.ID.isin(data_test.ID)]
dataA = data[data.ID.isin(data_train.ID)]

print (dataA.shape, dataA2.shape)
id_test = dataA2.ID.values

del dataA['ID']
del dataA2['ID']

# СОХРАНИТЬ ДАННЫЕ
data_tmp = dataA.copy()
data_tmp['y'] = y
data_tmp.to_csv('my_train__.csv', index=False)
dataA2.to_csv('my_test__.csv', index=False)
del data_tmp

category NUM_SOURCE (255722, 5)
category CREDIT_TYPE (255722, 7)
category CREDIT_ACTIVE (255722, 5)
category CREDIT_CURRENCY (255722, 4)
category CREDIT_FACILITY (255722, 6)
category DTIME_CREDIT_dayofweek (255722, 8)
category DTIME_CREDIT_ENDDATE_FACT_dayofweek (238467, 8)
category DTIME_CREDIT_UPDATE_dayofweek (255722, 8)
category TPD0 (255722, 10)
real CREDIT_DAY_OVERDUE (255722, 7)
real CNT_CREDIT_PROLONG (255722, 7)
real AMT_CREDIT_SUM (255722, 7)
real AMT_CREDIT_SUM_DEBT (255722, 7)
real AMT_CREDIT_SUM_LIMIT (255722, 7)
real AMT_CREDIT_SUM_OVERDUE (255722, 7)
real AMT_CREDIT_MAX_OVERDUE (255722, 7)
real CREDIT_DELAY5 (255722, 7)
real CREDIT_DELAY30 (255722, 7)
real CREDIT_DELAY60 (255722, 7)
real CREDIT_DELAY90 (255722, 7)
real CREDIT_DELAY_MORE (255722, 7)
real AMT_ANNUITY (255722, 7)
real DOLG (255722, 7)
real DELTA_CREDIT (255722, 7)
real DELTA_CREDIT2 (255722, 7)
real DELTA_ENDS (255722, 7)
real DTIME_CREDIT (255722, 7)
real DTIME_CREDIT_ENDDATE (255722, 7)
real DTIME_CREDIT_

In [11]:
# 2я версия

data_fm = [make_user_features(data_pre, user_features),
            make_cat_features(data_pre, cat_features + ['TPD0'], normalize=True),
            make_bin_features(data_pre, bin_features),
            make_real_features(data_pre, real_features),
            make_real_features(data_pre, realtime_features),
            make_real_features(data_pre, txt_features + txt_features2 + txt_features0),
            make_real_features(data_pre, txt_features3)
           ]

data = make_feature_matrix(data_fm, data_all.groupby('ID').size().reset_index()['ID'].values)

dataB2 = data[data.ID.isin(data_test.ID)]
dataB = data[data.ID.isin(data_train.ID)]
print (dataB.shape, dataB2.shape)

id_test2 = dataB2.ID.values
del dataB['ID']
del dataB2['ID']

#del data1['0c_SK_DATE_DECISION']
#del data2['0c_SK_DATE_DECISION']

# СОХРАНИТЬ ДАННЫЕ
data_tmp = dataB.copy()
data_tmp['y'] = y
data_tmp.to_csv('my_train2__.csv', index=False)
dataB2.to_csv('my_test2__.csv', index=False)

category NUM_SOURCE (255722, 5)
category CREDIT_TYPE (255722, 7)
category CREDIT_ACTIVE (255722, 5)
category CREDIT_CURRENCY (255722, 4)
category CREDIT_FACILITY (255722, 6)
category DTIME_CREDIT_dayofweek (255722, 8)
category DTIME_CREDIT_ENDDATE_FACT_dayofweek (238467, 8)
category DTIME_CREDIT_UPDATE_dayofweek (255722, 8)
category TPD0 (255722, 10)
real CREDIT_DAY_OVERDUE (255722, 7)
real CNT_CREDIT_PROLONG (255722, 7)
real AMT_CREDIT_SUM (255722, 7)
real AMT_CREDIT_SUM_DEBT (255722, 7)
real AMT_CREDIT_SUM_LIMIT (255722, 7)
real AMT_CREDIT_SUM_OVERDUE (255722, 7)
real AMT_CREDIT_MAX_OVERDUE (255722, 7)
real CREDIT_DELAY5 (255722, 7)
real CREDIT_DELAY30 (255722, 7)
real CREDIT_DELAY60 (255722, 7)
real CREDIT_DELAY90 (255722, 7)
real CREDIT_DELAY_MORE (255722, 7)
real AMT_ANNUITY (255722, 7)
real DOLG (255722, 7)
real DELTA_CREDIT (255722, 7)
real DELTA_CREDIT2 (255722, 7)
real DELTA_ENDS (255722, 7)
real DTIME_CREDIT (255722, 7)
real DTIME_CREDIT_ENDDATE (255722, 7)
real DTIME_CREDIT_

### Определение модели алгоритмов

смесь:

* 4 lgb.LGBMClassifier при разных параметрах
* 4 xgb.XGBClassifier при разных параметрах

In [39]:
from sklearn.base import BaseEstimator, ClassifierMixin
import lightgbm as lgb
import xgboost as xgb

class djLGB(BaseEstimator, ClassifierMixin):  
    """смесь lgb и xgb"""

    def __init__(self, seed=0, nest_lgb=1.0, nest_xgb=1.0, cbt=0.5, ss=0.5, alpha=0.5):
        """
        Инициализация
        seed - инициализация генератора псевдослучайных чисел
        nest_lgb, nest_xgb - сколько деревьев использовать (множитель)
        cbt, ss - процент признаков и объектов для сэмплирования
        alpha - коэффициент доверия XGB
        """
        print('LGB + XGB')
        self.models = [lgb.LGBMClassifier(num_leaves=2, learning_rate=0.07, n_estimators=int(1400*nest_lgb),
                                          colsample_bytree=cbt, subsample=ss,
                                          nthread=-1, random_state=0+seed),
                       lgb.LGBMClassifier(num_leaves=3, learning_rate=0.07, n_estimators=int(800*nest_lgb),
                                          colsample_bytree=cbt, subsample=ss,
                                          nthread=-1, random_state=1+seed),
                       lgb.LGBMClassifier(num_leaves=4, learning_rate=0.07, n_estimators=int(800*nest_lgb),
                                          colsample_bytree=cbt, subsample=ss,
                                          nthread=-1, random_state=2+seed),
                       lgb.LGBMClassifier(num_leaves=5, learning_rate=0.07, n_estimators=int(600*nest_lgb),
                                          colsample_bytree=cbt, subsample=ss,
                                          nthread=-1, random_state=3+seed,),
                       xgb.XGBClassifier(max_depth=1,
                                         learning_rate=0.1,
                                         n_estimators=int(800*nest_xgb),
                                         subsample=ss,
                                         colsample_bytree=cbt,
                                         nthread=-1,
                                         seed=0+seed),
                       xgb.XGBClassifier(max_depth=2,
                                         learning_rate=0.1,
                                         n_estimators=int(400*nest_xgb),
                                         subsample=ss,
                                         colsample_bytree=cbt,
                                         nthread=-1,
                                         seed=1+seed),
                       xgb.XGBClassifier(max_depth=3,
                                         learning_rate=0.1,
                                         n_estimators=int(200*nest_xgb),
                                         subsample=ss,
                                         colsample_bytree=cbt,
                                         nthread=-1,
                                         seed=2+seed),
                       xgb.XGBClassifier(max_depth=4,
                                         learning_rate=0.1,
                                         n_estimators=int(200*nest_xgb),
                                         subsample=ss,
                                         colsample_bytree=cbt,
                                         nthread=-1,
                                         seed=3+seed)
                      ]
        self.weights = [(1-alpha)*1, (1-alpha)*1, (1-alpha)*1, (1-alpha)*0.5, alpha*0.5, alpha*1, alpha*1.5, alpha*0.5]


    def fit(self, X, y=None):
        """
        обучение
        """
        for t, clf in enumerate(self.models):
            # print ('train', t)
            clf.fit(X, y)
        return self

    def predict(self, X):
        """
        определение вероятности
        """
        suma = 0.0
        for t, clf in enumerate(self.models):
            a = clf.predict_proba(X)[:, 1]
            suma += (self.weights[t] * a)
        return (suma / sum(self.weights))
            
    def predict_proba(self, X):
        """
        определение вероятности
        """        
        return (self.predict(X))

### Работа алгоритма

усреднение 15 описанных моделей

число деревьев в ансамблях взято побольше...

In [15]:
a_A = 0.0
a_B = 0.0

N = 2 # сколько усреднять 15

for t in range(N):
    clf = djLGB(seed=2000 + 10*t, nest_lgb=1.3, nest_xgb=1.3)
    clf.fit(dataA, y)
    a_A += clf.predict(dataA2)
    
    clf = djLGB(seed=(3000 + 10*t), nest_lgb=1.3, nest_xgb=1.3)
    clf.fit(dataB, y)
    a_B += clf.predict(dataB2)
    
a = (a_A + a_B) / 2

a = a - min(a)
a = a / max(a)

pd.DataFrame({'ID': id_test, 'Score': a}).to_csv('ridge_C04__.csv', index=False)

LGB + XGB
LGB + XGB
LGB + XGB
LGB + XGB


Придётся подождать, пока построятся все модели...